In this notebook I have done a survey on "FOOD" handles of Instagram by web scraping using Selenium. Same code can be be applied for other content/topics
(Change the search handle in 1.1 for results on other desired topics)

To use some other functionalities of this Bot refer to InstaBot- Part-I notebook.

### Instructions

1. Enter your Username and Password in 'Log in Instagram' section.

2. Please run the codes in sequence as given, as some functions are dependent.

3. Incase you feel the site has stopped processing the code, please have some patience as some components take time to load and code has been accordingly written to handle it.


## Creating Web Driver Session 

In [154]:
#importing selenium libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import time
from bs4 import BeautifulSoup

In [140]:
driver=webdriver.Chrome(executable_path='/home/gk-gunjan/Desktop/chromedriver')

In [141]:
def logIn(user_name,Password):
    driver.get('https://www.instagram.com/?hl=en')
    time.sleep(5)
    assert 'Instagram' in driver.title
    us=driver.find_element_by_name("username")
    us.send_keys(user_name)
    ps=driver.find_element_by_name("password")
    ps.send_keys(Password)
    bt=driver.find_element_by_xpath("//button[contains(@class,'y3zKF')]")
    bt.submit()
    time.sleep(5)

In [142]:
def login_process():   
    try :
        #dont save information
        bt=driver.find_element_by_xpath("//button[contains(@class,'sqdOP yWX7d    y3zKF     ')]")
        bt.click()
    except NoSuchElementException:
        pass
     
        
    try:
        # dont turn on notification
        bt=driver.find_element_by_xpath("//button[contains(@class,'aOOlW   HoLwm ')]")
        bt.click() 
        driver.maximize_window()
    except NoSuchElementException:
        pass

In [143]:
#Enter your instagram username and password inplace of sample username and sample password
logIn("gunjan.photogallery","Najnug.28")

In [144]:
#for not saving info and turning off notifications
login_process()

## 1.1 Exracting top 5 handles with highest followers

### Search 

In [145]:
# #gives a list of first ten instagram handles  on searching
# def search(text):
#     search=driver.find_element_by_xpath("//div[contains(@class,'LWmhU _0aCwM')]/input")
#     search.clear()
#     search.send_keys(text)
#     time.sleep(5)
#     li=[]
#     drop=driver.find_elements_by_class_name("uL8Hv")
#     for i in drop:
#         x=i.text
#         if x[0] != "#":
#             li.append(x)
#     return li[8:18]

In [146]:
# #enter your desired search key
# top_ten=search("food")

In [148]:
# #To open any instagram profile or handle- enter the username in place of handle
# def open_profile(handle):
#     driver.get('https://www.instagram.com/?hl=en')
#     time.sleep(5)
#     #search textfield
#     search=driver.find_element_by_xpath("//div[contains(@class,'LWmhU _0aCwM')]/input")
#     search.clear()
#     search.send_keys(handle)
#     time.sleep(5)
    
#     cl=driver.find_element_by_class_name('-qQT3')
#     cl.click()

### Handles with highest followers

In [149]:
# #gives top 5 handles with highest followers from the list 'ten'
# def top_follow():
#     d={}
#     for i in top_ten:
#         open_profile(i)
#         time.sleep(9)
#         followers = driver.find_elements_by_class_name('g47SY')[1].get_attribute('title') # extracting followers of each handle
#         d[i] = int(followers.replace(',','')) # converting to integer value
    
#     #sorts the dictionary by followers
#     sorted_d = sorted(d.items(), key = lambda kv:kv[1], reverse = True)[:5] # sorting the by number of followers 
#     return sorted_d

In [155]:
#fetching names of the Instagram Handles that are displayed in list after typing “food” 
   
def search_and_open(profile):
    driver.get("https://www.instagram.com/"+profile)
    
   
 #finding number of followers  
def find_number_of_followers(profile):
    search_and_open(profile)
    wait = WebDriverWait(driver, 10)
    wait2= wait.until(EC.presence_of_element_located((By.CLASS_NAME,"-nal3"))) 
    data = driver.page_source
    html_data = BeautifulSoup(data, 'html.parser')
    n= html_data.find_all(class_ = '-nal3')
    
    num_f=n[1].span["title"]

    
    return num_f


In [157]:
search=driver.find_element_by_xpath('//input[contains(@class,"XTCLo")]')
search.clear()
search.send_keys("food")
time.sleep(3)

l=driver.find_elements_by_class_name("uL8Hv")
l=l[8:18]
insta_handles=[]
for i in l:
    insta_handles.append(i.get_attribute('innerHTML').strip('#'))
   
    
d={}
for i in insta_handles:
    num_followers=find_number_of_followers((i))
    
    num_followers=num_followers.replace(",","")
    d[i]=int(num_followers)

#finding top 5
sort_d={k: v for k,v in sorted(d.items(), key=lambda item:item[1],reverse=True)}
c=0
print("The top 5 which have the highest number of followers :-")
for i in sort_d:
    if(c==5):
        break
    
    print(c+1,i,"-->",sort_d[i])
    c+=1

The top 5 which have the highest number of followers :-
1 dilsefoodie --> 764179
2 foodie_incarnate --> 308020
3 foodtalkindia --> 304623
4 thisisdelhi --> 165167
5 food --> 116884


## 1.2

In [159]:
insta_handles

['foodtalkindia',
 'dilsefoodie',
 'pandareviewz',
 'mccainfoods_india',
 'thisisdelhi',
 'delhifoodguide',
 'food',
 'delhieater',
 'food_lunatic',
 'foodie_incarnate']

In [ ]:
print("Top 5 handles (w.r.t followers) : ")
print()

print(" ", "Handle","      ","Followers")

five=[]
for i,j in sorted_top:
    print(i," : ",j)
    five.append(i)

In [161]:
sort_d

{'dilsefoodie': 764179,
 'foodie_incarnate': 308020,
 'foodtalkindia': 304623,
 'thisisdelhi': 165167,
 'food': 116884,
 'food_lunatic': 108803,
 'delhifoodguide': 102487,
 'pandareviewz': 25693,
 'delhieater': 24096,
 'mccainfoods_india': 6453}